# Sagemaker Tutorial Series

## Tutorial - 2 Create Rest API for Sagemaker Endpoint using Lambda & API Gateway

### Let's divide the workload:

1. Deploy the Sagemaker Endpoint. 
2. Create Lambda Function
3. Create Rest API in API Gateway
4. Deploy API
5. Test API using Postman and Python Script

### 1. Deploy the Sagemaker Endpoint. 
- Refer Tutorial-1 Notebook, and deploy endpoint from there. Please use below training script as reference. This contains custom created inference function. In last tutorial series we have used default inference function. 

In [ ]:
%%writefile script.py


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# inference functions ---------------

def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(input_data, model):
    if type(input_data) != str:
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

### 2. Create Lambda Function (Example Code)
- Follow Tutorial video for more clarity.

In [ ]:
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime= boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
    payload = json.loads(json.dumps(event))
    payload_data = str(payload['body'])
    print(payload_data)
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                      ContentType='text/csv',
                                      Body=payload_data)
    result = json.loads(response['Body'].read().decode())
    preds = {"Prediction": result}
    response_dict = {
          "statusCode": 200,
          "body": json.dumps(preds)
                }
    return response_dict

- Attach this policy to Lambda Execution Role

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "sagemaker:InvokeEndpoint",
            "Resource": "*"
        }
    ]
}

### 3. Create Rest API in API Gateway
- Follow Tutorial video for more clarity.

### 4. Deploy API
- In this step we are deploying our Rest API into the Stage, we will get one API URL. 

### 5. Test API using Postman and Python Script
- Our API is running in stage, now we need to check API.
- We can use POSTMAN and Python Script both for checking the API results. 

In [5]:
import requests

url = "------YOUR API URL--------"

payload = "1454.0,1.0,0.5,1.0,1.0,0.0,34.0,0.7,83.0,4.0,3.0,250.0,1033.0,3419.0,7.0,5.0,5.0,1.0,1.0,0.0"

headers = {
  'Content-Type': 'text/csv'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"Prediction": [3]}


### Don't forget to Subscribe Machine Learning Hub YouTube Channel. 